# 실험 12-01 공통 드레인 증폭기

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1-1 결과 분석 (실험 과정 4)

### 회로도

* R1 = 500k Ohmn
* R2 = 750k Ohm
* RS = 0.15k Ohm
* C1 = 10u F

![Markdown-mark](../experiment/circuit_01_01_01.jpeg)
![Markdown-mark](../experiment/circuit_01_01_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
V_DD_V      = 12
R_S_kOhm    = 0.15
V_Tn_V      = 1.990

V_GQ_V     = ?
V_SQ_V     = ?

V_GSQ_V     = V_GQ_V - V_SQ_V
V_DSQ_V     = V_DD_V - V_SQ_V

I_DQ_mA     = V_SQ_V / R_S_kOhm

g_m_mA_V    = (2 * I_DQ_mA) / (V_GSQ_V - V_Tn_V)

print_value('V_GQ_V     ', V_GQ_V)
print_value('V_SQ_V     ', V_SQ_V)
print('')


print_value('V_GSQ_V    ', V_GSQ_V)
print_value('I_DQ_mA    ', I_DQ_mA)
print_value('V_DSQ_V    ', V_DSQ_V)
print_value('g_m_mA_V   ', g_m_mA_V)

*******************************************************************************
## 실험 1-2 결과 분석 (실험 과정 6)

## 회로도

![Markdown-mark](../experiment/circuit_01_01_01.jpeg)
![Markdown-mark](../experiment/circuit_01_02_02.jpeg)

### 1. 실험 결과를 가져온다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/m_01_02_01.csv')

### 2. 실험 결과에서 $V_{S}$, $V_{O}$를 가져온다.

In [ ]:
TIME_ms_a = data['TIME'] * 1000.
V_S_V_a = data['Sample CH1']
V_S_V_a = savgol_filter(V_S_V_a, 100, 2)
V_O_V_a = data['Sample CH2']
V_O_V_a = savgol_filter(V_O_V_a, 100, 2)

### 3. $V_{S}$와 $V_{O}$의 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_S_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([-1.1, 1.1])
draw_plot(TIME_ms_a, V_S_V_a, '(R_S_kOhm=%1.3f)' % R_S_kOhm, 0, marker_num=5)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_O_V')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.2, 2.0])
plt.ylim([V_O_V_a.mean() - 1.1, V_O_V_a.mean() + 1.1])
draw_plot(TIME_ms_a, V_O_V_a, '(R_S_kOhm=%1.3f)' % R_S_kOhm, 0, marker_num=5)
plt.show()

### 4. $V_{S}$와 $V_{O}$의 첨두-첨두 값을 구한다.

In [ ]:
V_S_V_pp = V_S_V_a.max() - V_S_V_a.min()
V_O_V_pp = V_O_V_a.max() - V_O_V_a.min()

print_value('R_S_kOhm       ', R_S_kOhm)
print_value('V_S_V_pp       ', V_S_V_pp)
print_value('V_O_V_pp       ', V_O_V_pp)

### 5. 전압이득 $A_{v}$ 값을 구한다.

In [ ]:
A_v_meas = V_O_V_pp / V_S_V_pp

print_value('R_S_kOhm       ', R_S_kOhm)
print_value('A_v_meas       ', A_v_meas)

### 6. 식 (12.2)을 사용하여 전압이득 $A_{v}$ 값을 구한다.

In [ ]:
# g_m_mA_V_sim = ?

R_S_kOhm = 0.15
g_m_mA_V_sim = 159.069

A_v_cal = (g_m_mA_V_sim * R_S_kOhm) / (1 + g_m_mA_V_sim * R_S_kOhm)

print_value('R_S_kOhm   ', R_S_kOhm)
print_value('A_v_cal    ', A_v_cal)